<a href="https://colab.research.google.com/github/Mahmuda-104/Thesis/blob/main/BiLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import os
from scipy.io import loadmat

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
data_path = '/content/drive/My Drive/Data'

In [4]:
import os
import scipy.io
import numpy as np
from scipy import signal
fs_original = 500
fs_new = 250

control_data = []
control_path = os.path.join(data_path, 'Control')
for foldername in os.listdir(control_path):
    subfolder_path = os.path.join(control_path, foldername)
    control_data.append(subfolder_path)

Control30 = []
for control_data_path in control_data:
    n_epochs = 35 #len([f for f in os.listdir(control_data_path) if f.endswith('.mat')])
    control_arr = []
    for i in range(1, n_epochs+1):
        epoch_path = f"{control_data_path}/trial{i}.mat"
        mat_data = scipy.io.loadmat(epoch_path)
        mat_data1 = mat_data["trialData_i"]

        # Downsampling
        num_samples_original = mat_data1.shape[-1]
        num_samples_new = int(num_samples_original * fs_new / fs_original)
        downsampled_data = signal.resample(mat_data1, num_samples_new, axis=-1)

        control_arr.append(np.array(downsampled_data))

    Control30.append(np.array(control_arr))

Control30 = np.array(Control30)
print( Control30[0].shape)


(35, 60, 1000)


In [5]:
import os
import scipy.io
import numpy as np
from scipy import signal

fs_original = 500
fs_new = 250

concussed_data = []
concussed_path = os.path.join(data_path, 'Concussed')
for foldername in os.listdir(concussed_path):
    subfolder_path = os.path.join(concussed_path, foldername)
    concussed_data.append(subfolder_path)

Concussed52 = []
for concussed_data_path in concussed_data:
    n_epochs = 35 #len([f for f in os.listdir(control_data_path) if f.endswith('.mat')])
    concussed_arr = []
    for i in range(1, n_epochs+1):
        epoch_path = f"{concussed_data_path}/trial{i}.mat"
        mat_data = scipy.io.loadmat(epoch_path)
        mat_data1 = mat_data["trialData_i"]

        # Downsampling
        num_samples_original = mat_data1.shape[-1]
        num_samples_new = int(num_samples_original * fs_new / fs_original)
        downsampled_data = signal.resample(mat_data1, num_samples_new, axis=-1)

        concussed_arr.append(np.array(downsampled_data))

    Concussed52.append(np.array(concussed_arr))

Concussed52 = np.array(Concussed52)
print( Concussed52[0].shape)


(35, 60, 1000)


In [6]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Step 1: Data Preparation
# Combine control and concussed data
data_combined = np.concatenate((Control30, Concussed52), axis=0)

# Create labels for binary classification
control_labels = np.zeros(Control30.shape[0])
concussed_labels = np.ones(Concussed52.shape[0])
labels_combined = np.concatenate((control_labels, concussed_labels))

# Step 2: Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(data_combined, labels_combined, test_size=0.2, random_state=42)

# Step 3: Normalize the data
scaler = MinMaxScaler()
x_train_normalized = scaler.fit_transform(x_train.reshape(x_train.shape[0], -1))
x_test_normalized = scaler.transform(x_test.reshape(x_test.shape[0], -1))

# Reshape the normalized data back to its original shape
x_train_normalized = x_train_normalized.reshape(x_train.shape)
x_test_normalized = x_test_normalized.reshape(x_test.shape)

# Verify the shapes of the training and testing sets
print("x_train shape:", x_train_normalized.shape)
print("y_train shape:", y_train.shape)
print("x_test shape:", x_test_normalized.shape)
print("y_test shape:", y_test.shape)


x_train shape: (67, 35, 60, 1000)
y_train shape: (67,)
x_test shape: (17, 35, 60, 1000)
y_test shape: (17,)


In [7]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, GRU, Dense, Dropout, Reshape, Bidirectional
from keras.callbacks import EarlyStopping

# Reshape the data
x_train_reshaped = x_train.reshape(x_train.shape[0], -1)
x_test_reshaped = x_test.reshape(x_test.shape[0], -1)

# Standardize the data
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train_reshaped)
x_test_scaled = scaler.transform(x_test_reshaped)

# Convert the labels to categorical (one-hot encoding)
from keras.utils import to_categorical
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

# Reshape the flattened data back to 3D shape
x_train_scaled = x_train_scaled.reshape(x_train.shape[0], x_train.shape[1], -1)
x_test_scaled = x_test_scaled.reshape(x_test.shape[0], x_test.shape[1], -1)

# Build the model architecture
model = Sequential()
model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=(x_train.shape[1], x_train_scaled.shape[2])))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(units=256, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(units=64, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(GRU(units=64, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(GRU(units=32)))
model.add(Dropout(0.2))
model.add(Dense(units=16, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(x_train_scaled, y_train_categorical, validation_data=(x_test_scaled, y_test_categorical),
                    epochs=200, batch_size=256, callbacks=[early_stopping])

# Evaluate the model on the test set
loss, accuracy = model.evaluate(x_test_scaled, y_test_categorical)
print('Test loss:', loss)
print('Test accuracy:', accuracy)


Epoch 1/200
1/1 [==============================] - 64s 64s/step - loss: 0.6720 - accuracy: 0.6567 - val_loss: 0.6316 - val_accuracy: 0.7059
Epoch 2/200
1/1 [==============================] - 28s 28s/step - loss: 0.4868 - accuracy: 0.7761 - val_loss: 0.6589 - val_accuracy: 0.5882
Epoch 3/200
1/1 [==============================] - 24s 24s/step - loss: 0.2270 - accuracy: 1.0000 - val_loss: 0.7577 - val_accuracy: 0.6471
Epoch 4/200
1/1 [==============================] - 21s 21s/step - loss: 0.0994 - accuracy: 1.0000 - val_loss: 0.8947 - val_accuracy: 0.6471
Epoch 5/200
1/1 [==============================] - 22s 22s/step - loss: 0.0488 - accuracy: 1.0000 - val_loss: 1.0530 - val_accuracy: 0.5882
Epoch 6/200
1/1 [==============================] - 23s 23s/step - loss: 0.0209 - accuracy: 1.0000 - val_loss: 1.2106 - val_accuracy: 0.6471
Epoch 7/200
1/1 [==============================] - 24s 24s/step - loss: 0.0117 - accuracy: 1.0000 - val_loss: 1.3748 - val_accuracy: 0.6471
Epoch 8/200
1/1 [===

In [9]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report


y_pred_probs = model.predict(x_test_scaled)

y_pred = np.argmax(y_pred_probs, axis=1)

y_test_single = np.argmax(y_test_categorical, axis=1)

cm = confusion_matrix(y_test_single, y_pred)

report = classification_report(y_test_single, y_pred)

print("Confusion Matrix:")
print(cm)
print("\nClassification Report:")
print(report)


1/1 [==============================] - 2s 2s/step
Confusion Matrix:
[[ 2  4]
 [ 1 10]]

Classification Report:
              precision    recall  f1-score   support

           0       0.67      0.33      0.44         6
           1       0.71      0.91      0.80        11

    accuracy                           0.71        17
   macro avg       0.69      0.62      0.62        17
weighted avg       0.70      0.71      0.67        17

